# Import libraries

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.autograd import grad
import numpy as np
from copy import deepcopy
import pickle
import matplotlib.pyplot as plt
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
from natsort import natsorted

In [ ]:
device

device(type='cuda', index=0)

# Model definition

In [ ]:
class SurrogateModel(nn.Module):
  def __init__(self, name=None):        
    super().__init__()
    
    # NN module
    self.stack = nn.Sequential(
        nn.Linear(2, 20),
        nn.Tanh(),
        nn.Linear(20,20),
        nn.Tanh(),
        nn.Linear(20,20),
        nn.Tanh(),
        nn.Linear(20,20),
        nn.Tanh(),
        nn.Linear(20,20),
        nn.Tanh(),
        nn.Linear(20,20),
        nn.Tanh(),
        nn.Linear(20,20),
        nn.ReLU(),
        #nn.Tanh(),
        nn.Linear(20,20),
        nn.ReLU(),
        #nn.Tanh(),
        nn.Linear(20,20),
        nn.ReLU(),
        #nn.Tanh(),
        nn.Linear(20,1)
    )

  # Forward calculation
  def forward(self, x):
    return self.stack(x)

## Physics : Terzaghi's equation 

In [ ]:
class Consolidation_Equation():
  def __init__(self, model, z_obs, t_obs, z_phy, t_phy, upper, lower, cv):
    self.model = model
    self.z_obs = z_obs
    self.t_obs = t_obs
    self.z_phy = z_phy
    self.t_phy = t_phy
    self.cv = cv

    self.z_max = upper[0] 
    self.z_min = lower[0] 
    self.t_max = upper[1] 
    self.t_min = lower[1] 
    self.e_max = upper[2]
    self.e_min = lower[2]

    self.z_sf = self.z_max - self.z_min 
    self.t_sf = self.t_max - self.t_min 
    self.e_sf = self.e_max - self.e_min 

  # Prediction
  def predict(self, z, t):

    # Store input into a tensor
    z_sd = (z - self.z_min)/self.z_sf
    t_sd = (t - self.t_min)/self.t_sf
    input = torch.cat((z_sd, t_sd), 1)

    e = self.e_sf * self.model(input)

    return e

  # Governing Equation
  def GE(self, z, t, e, cv):

    e_t = self.t_sf * grad(e.sum(), t, create_graph=True)[0] 
    e_z = self.z_sf * grad(e.sum(), z, create_graph=True)[0]

    e_zz = self.z_sf * grad(e_z.sum(), z, create_graph=True)[0] 
    

    # Governing Equation : Darcy's law
    f_e = e_t - self.t_sf/self.z_sf/self.z_sf * self.cv * e_zz 

    return f_e
    
  def train(self, N_Iter, N_out, optimizer, weight=0.5, eps=1e-10):

    N_obs = self.z_obs.size()[0]
    N_phy = self.z_phy.size()[0]

    for iter in range(N_Iter):

      if N_phy > 0:
        # Observation + Physics
        # Predictions
        e_pre = self.predict(self.z_obs, self.t_obs)
        
        # Governing Equations
        e_test = self.predict(self.z_phy, self.t_phy)         
        f_e = self.GE(self.z_phy, self.t_phy, e_test, self.cv) 

        # Loss functions
        # vs. observation
        loss_obs = (torch.square(e_pre - e_obs).sum())/N_obs

        # vs. physics
        loss_phy = (torch.square(f_e).sum())/N_phy

        # Total error
        loss = weight*loss_obs + (1.0-weight)*loss_phy

        
      else:
        # Observation only
        # Predictions
        e_pre = self.predict(self.z_obs, self.t_obs)

        # Loss functions
        # only vs. observation
        loss = (torch.square(e_pre - e_obs).sum())/N_obs

      # Backpropagation
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      if loss.item() < eps:
        print('break')
        break

      if iter % N_out == 0:
        print(f"Loss: {loss.item():>7f}  [{iter:>5d}/{N_Iter:>5d}]")

    print(f"Loss: {loss.item():>7f}  [{N_Iter:>5d}/{N_Iter:>5d}]")
    if N_phy > 0:
      print(f" obs : {loss_obs.item():>5e}")
      print(f" phy : {loss_phy.item():>5e}")
    print(f" cv : {self.cv.item():>5e}")

    return loss.item(), loss_obs.item(), loss_phy.item()

  def test(self, z_test, t_test, e_test):
 
    N_test = z_test.size()[0]
    
    e_pre = self.predict(z_test, t_test)
    error = (torch.square(e_pre - e_test).sum() )/N_test

    print(f"Test Error: {error.item():>7f}")

    return error.item()

# Load simulation data

## Mount Google Drive in Google Colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive')
print(os.getcwd())

/content/drive/MyDrive/20210810_PINN/20211208/e/data/PINN


## Load data

In [ ]:
import scipy.io
import glob
import os
from scipy.interpolate import griddata

files_obs = glob.glob('10days/*')
files_obs = natsorted(files_obs)

data_obs = np.empty((0,2))

for file in files_obs:
  filename = os.path.basename(file)
  temp_obs = np.loadtxt('10days/' + filename, ndmin=2, delimiter=" ")
  data_obs = np.append(data_obs, temp_obs, axis=0)

# Observation data
t_org = np.arange(len(files_obs)).reshape(len(files_obs),1) * 3600

ZZ = data_obs[:,0].reshape((-1, len(files_obs)))
EE = data_obs[:,1].reshape((-1, len(files_obs)))

N = ZZ.shape[0]
T = t_org.shape[0]

TT = np.tile(t_org, (1,N)).T

z = ZZ.flatten()[:,None] # NT x 1
ttt = TT.flatten()[None,:] # NT x 1
t = (np.sort(ttt)).T
e = EE.flatten()[:,None] # NT x 1

# Physics data
t_p1 = np.arange(0.,8640001., 3600.).reshape(-1,1)
t_p = np.repeat(t_p1,501).reshape(-1,1).flatten()[:,None] 
z_p1 = np.arange(0.,5.01,0.01).reshape(-1,1)
z_p = []
for i in range(2401):
  z_p.append(z_p1)
z_p = np.array(z_p).reshape(-1,1).flatten()[:,None] 

N_p = z_p1.shape[0]
T_p = t_p1.shape[0]

# Expected upper and lower bounds of input for surrogate model
upper = np.array([z_p.max(), t_p.max(), e_o.max()])
lower = np.array([z_p.min(), t_p.min(), e_o.min()])

true = np.loadtxt('true/e_8640000.txt', delimiter = " ", dtype = 'float32')
t_true = np.full((501,1), 8640000., dtype = 'float32')
z_true = true[:,0].flatten()[:,None]
e_true = true[:,1].flatten()[:,None]

# Observation and Test dataset (N_obs = 10,000, N_test = 501)

In [ ]:
# Number of observation and test points
N_obs = 10000
N_test = 501

np.random.seed(123453)

# Index list for observation and validation data
idx_obs = np.random.choice(N*T, N_obs, replace=False)

# Observation points for training
z_obs = Variable(torch.tensor(z[idx_obs,:].astype(np.float32)), requires_grad=True).to(device)
t_obs = Variable(torch.tensor(t[idx_obs,:].astype(np.float32)), requires_grad=True).to(device)

# Observed velocity field for training
e_obs = torch.tensor(e[idx_obs,:].astype(np.float32)).to(device)

# Collocation points for validation
z_test = torch.tensor(z_true).to(device) 
t_test = torch.tensor(t_true).to(device) 

# Observed velocity field for validation
e_test = torch.tensor(e_true).to(device)

# Train using observation and physics (N_phy = N_obs = 10,000, phy =! obs)

In [ ]:
# Model input and parameters
N_phy = 10000 

np.random.seed(123453)

# Index list for observation and validation data
idx_phy = np.random.choice(N_p*T_p, N_phy, replace=False)

# Points for training
z_phy = Variable(torch.tensor(z_p[idx_phy,:].astype(np.float32)), requires_grad=True).to(device)
t_phy = Variable(torch.tensor(t_p[idx_phy,:].astype(np.float32)), requires_grad=True).to(device)

# Collocation points for physics
#z_phy = z_obs
#t_phy = t_obs

##Noise = 0.0

### Allocation of variables and model parameters

In [ ]:
# Model input and parameters

noise_level = 0.0

# Observed velocity field for training
e_obs = torch.tensor(e[idx_obs,:].astype(np.float32)).to(device)

# Additional noise
e_noise = torch.tensor(noise_level * (e.max()-e.min()) * np.random.randn(N_obs,1)).to(device)

e_obs = e_obs + e_noise

# Model parameters in governing equation
cv = torch.tensor(5.2 * pow(10, -7)).to(device)

# tensors to normalize input for NN surrogate model
ub = torch.tensor(upper.astype(np.float32)).to(device)
lb = torch.tensor(lower.astype(np.float32)).to(device)

### No.1: Training loop for model parameters in surrogate model

In [ ]:
N_epoch = 40
N_Iter = 1000
N_out = 200

# Model definition
torch.manual_seed(123453)
np.random.seed(123453)

surrogate = SurrogateModel().to(device)
physics = Consolidation_Equation(surrogate,
                                 z_obs, t_obs,
                                 z_phy, t_phy,
                                 upper, lower,
                                 cv)


optimizer = torch.optim.Adam([
                              {'params': surrogate.parameters()},
                             ], lr=1e-3)

train_loss = np.array([])
test_error = np.array([])

loss_obs = np.array([])
loss_phy = np.array([])

min_error = 1e15

for epoch in range(N_epoch):
  print(f"Epoch: {epoch:>5d}")
  loss = physics.train(N_Iter, N_out, optimizer, eps=1e-15)
  error = physics.test(z_test, t_test, e_test)
  if error < min_error:
    min_error = error
    min_surrogate = deepcopy(surrogate)
  print("")
  #print(loss)

  train_loss = np.append(train_loss, loss[0])
  test_error = np.append(test_error, error)

  loss_obs = np.append(loss_obs, loss[1])
  loss_phy = np.append(loss_phy, loss[2])

  e_check = physics.predict(z_obs, t_obs)
  f_e = physics.GE(z_obs, t_obs, e_check, cv)
  e_predict = physics.predict(z_test, t_test)
  
  path = 'results/'
  np.savetxt(path + str(epoch) + 'e_pre.txt', e_check.to('cpu').detach())
  np.savetxt(path + str(epoch) + 'e_obs.txt', e_obs.to('cpu').detach())
  np.savetxt(path + str(epoch) + 'z_obs.txt', z_obs.to('cpu').detach())
  np.savetxt(path + str(epoch) + 't_obs.txt', t_obs.to('cpu').detach())
  np.savetxt(path + str(epoch) + 'f_e.txt', f_e.to('cpu').detach())
  np.savetxt(path + str(epoch) + 'e_test_true.txt', e_test.to('cpu').detach())
  np.savetxt(path + str(epoch) + 'e_test_pre.txt', e_predict.to('cpu').detach())  
  np.savetxt(path + str(epoch) + 'e_0_obs.txt', e_0_obs.to('cpu').detach())  

#### Minimum Estimation Error

In [ ]:
np.savetxt(path + 'train_loss.txt', train_loss) 
np.savetxt(path + 'test_error.txt', test_error) 
np.savetxt(path + 'loss_obs.txt', loss_obs) 
np.savetxt(path + 'loss_phy.txt', loss_phy) 
np.savetxt(path + 'loss_bc.txt', loss_bc) 

In [ ]:
epoch = np.arange(N_epoch)
fig = plt.figure(figsize=(10, 6))
ax1 = fig.add_subplot(111)
ax1.plot(epoch, train_loss, '-', color=red, label='train loss')


ax2 = ax1.twinx()
ax2.plot(epoch, test_error, '-', color=blue, label='test error')

plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
#plt.ylim(0, 4e-4)
ax1.set_xlabel('Epoch', fontsize=15)
ax1.set_ylabel('Loss', fontsize=15)
ax2.set_ylabel('Error', fontsize=15)

ax1.ticklabel_format(axis="y", scilimits=(0,0), useMathText=True)
ax2.ticklabel_format(axis="y", scilimits=(0,0), useMathText=True)

h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()

#ax1.legend(h1+h2, l1+l2, loc='upper right', bbox_to_anchor=(0.18, 0.98), fontsize=10)
ax1.legend(h1+h2, l1+l2, loc='upper right', bbox_to_anchor=(0.98, 0.98), fontsize=10)

#fig.subplots_adjust(left=0, right=1, bottom=0, top=1)
plt.savefig(path + 'loss_error_' + str(min_filenum_u) + 'epoch_' + str('{:.3e}'.format(test_error_u[min_filenum_u])) +  '.svg')
subprocess.call('inkscape sample.svg -M sample.emf',shell=True)
plt.show()

In [ ]:
min_filenum_e = np.argmin(test_error)
e_min_loss = np.loadtxt(path + str(min_filenum_e) + 'e_test_pre.txt')
depth=np.arange(0.,5.01,0.01).reshape(-1,1)

####################### strain #######################
fig = plt.figure(figsize=(6, 10))
ax = fig.add_subplot(111)
ax.plot(e_min_loss, depth, '-', color=black, label='Predicted strain') #, linestyle="dashed")
#ax.plot(e_min_loss_uepoch, depth, '-', color=black, label='Predicted strain', linestyle="dashdot")
ax.plot(e_true, depth, '-', color=gray, label='True')
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

ax.set_xlabel('Strain', fontsize=15)
ax.set_ylabel('Depth (m)', fontsize=15)

ax.invert_yaxis()

ax.ticklabel_format(axis="y", scilimits=(0,0), useMathText=True)

#plt.legend(bbox_to_anchor=(0.42, 0.07), loc='upper right', borderaxespad=1, fontsize=10)
plt.legend(bbox_to_anchor=(0.43, 1.0), loc='upper right', borderaxespad=1, fontsize=10)

plt.savefig(path + 'strain_' + str(min_filenum_e) + 'epoch_' + str('{:.6f}'.format(settlement_pre)) + '.svg')
#subprocess.call('inkscape sample.svg -M sample.emf',shell=True)
plt.show()